# Выпускной проект. SQL

__План (декомпозиция) анализа данных:__  

1. [Введение.](#intro)  
    1.1. [Описание проекта.](#project)  
    1.2. [Цель проекта.](#purpose)  
    1.3. [Описание данных.](#data)  
2. [Изучение общей информации.](#read_and_study)  
    2.1. [Импорт библиотек и установка параметров доступа к базе данных.](#import)  
    2.2. [Рассмотрение общей информации.](#info)  
3. [Выполнение заданий.](#tasks)  
    3.1. [Количество книг, вышедших после 1 января 2000 года.](#books_count)  
    3.2. [Количество обзоров и средняя оценка для каждой книги.](#reviews_count)  
    3.3. [Определение издательства, которое выпустило наибольшее число книг толще 50 страниц.](#publishing_house)  
    3.4. [Определение автора с самой высокой средней оценкой книг.](#author)  
    3.5. [Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.](#users_reviews)  
4. [Общий вывод.](#conclusion)  

##  <a id="intro">Введение.</a>

### <a id="project">Описание проекта.</a>

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. В представленной базе данных — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### <a id="purpose">Цель проекта.</a>

Проанализировать базу данных купленного сервиса для чтения книг по подписке и сформулировать ценностное предложение для нового продукта.

### <a id="data">Описание данных.</a>

Таблица `books` содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors` содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers` содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings` содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

##  <a id="read_and_study">Изучение общей информации.</a>

### <a id="import">Импорт библиотек и установка параметров доступа к базе данных.</a>

In [1]:
# Устанавливаем необходимые в проекте библиотеки
!pip install sqlalchemy
!pip install psycopg2-binary

In [2]:
# Импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [3]:
# Устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# Сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### <a id="info">Рассмотрение общей информации.</a>

In [4]:
# Определим функцию для предварительного исследования таблиц
def info(table):
    display(pd.io.sql.read_sql(sql=text('''SELECT * FROM {} LIMIT 5'''.format(table)), con = engine.connect()))
    pd.io.sql.read_sql(sql=text('''SELECT * FROM {}'''.format(table)), con = engine.connect()).info()
    display(pd.io.sql.read_sql(sql=text('''SELECT * FROM {}'''.format(table)), con = engine.connect()).describe())

Применяя функцию `info` рассмотрим общую информацию по таблицам.

In [5]:
df_list = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for df in df_list:
    print(f'Общая информация по таблице `{df}`:')
    info(df)
    print()

Общая информация по таблице `books`:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000



Общая информация по таблице `authors`:


...

,rating_id,book_id,rating
count,6456.000000,6456.000000,6456.000000
mean,3228.500000,510.574195,3.928284
std,1863.831001,284.141636,0.943303
min,1.000000,1.000000,1.000000
25%,1614.750000,291.000000,3.000000
50%,3228.500000,506.000000,4.000000
75%,4842.250000,750.000000,5.000000
max,6456.000000,1000.000000,5.000000



Общая информация по таблице `reviews`:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id
count,2793.000000,2793.000000
mean,1397.000000,504.693161
std,806.413976,288.472931
min,1.000000,1.000000
25%,699.000000,259.000000
50%,1397.000000,505.000000
75%,2095.000000,753.000000
max,2793.000000,1000.000000


__Промежуточный вывод:__ импортировали библиотеки, установили параметры доступа к базе данных и изучили общую информацию пяти таблиц: `books`, `authors`, `publishers`, `ratings`, `reviews`. Видим, что пропуски отсутствуют, типы данных соответствуют значениям.

##  <a id="tasks">Выполнение заданий.</a>

###  <a id="books_count">Количество книг, вышедших после 1 января 2000 года.</a>

In [6]:
query = '''
            SELECT COUNT(book_id) AS books_count
            FROM books
            WHERE publication_date>=DATE('2000-01-01')
        '''

In [7]:
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,books_count
0,821


После 1 января 2000 года вышла 821 книга.

###  <a id="reviews_count">Количество обзоров и средняя оценка для каждой книги.</a>

In [8]:
query = '''
            WITH t1 AS (
            SELECT
                book_id
                , COUNT(review_id) AS review_count
            FROM reviews
            GROUP BY 1
            ),
            t2 AS (
            SELECT
                book_id
                , AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY 1
            )
            SELECT
                title
                , SUM(review_count) AS review_count
                , AVG(avg_rating) AS avg_rating
            FROM books AS b
            LEFT JOIN t1 on b.book_id = t1.book_id
            LEFT JOIN t2 on b.book_id = t2.book_id
            GROUP BY 1
        '''

In [9]:
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,title,review_count,avg_rating
0,The Count of Monte Cristo,5.0,4.217391
1,Count Zero (Sprawl #2),2.0,2.500000
2,The Botany of Desire: A Plant's-Eye View of th...,2.0,3.500000
3,The Poisonwood Bible,5.0,4.363636
4,The Canterbury Tales,3.0,3.333333
...,...,...,...
994,In the Heart of the Sea: The Tragedy of the Wh...,3.0,3.333333
995,Of Love and Other Demons,2.0,4.500000
996,Welcome to Temptation (Dempseys #1),2.0,5.000000
997,World's End (The Sandman #8),2.0,4.500000


###  <a id="publishing_house">Определение издательства, которое выпустило наибольшее число книг толще 50 страниц.</a>

Исключим из анализа брошюры.

In [10]:
query = ''' 
            SELECT
                publisher
                , COUNT(book_id) as books_count 
            FROM books AS b
            LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1            
        '''

In [11]:
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,publisher,books_count
0,Penguin Books,42


Наибольшее число книг толщец 50 страниц выпустило издательство `Penguin Books` - 42 книги.

###  <a id="author">Определение автора с самой высокой средней оценкой книг.</a>

Будем учитывать только книги с 50 и более оценками.

In [12]:
query = '''
            WITH t1 AS (
            SELECT
                book_id
                , COUNT(rating) AS rating_count
                , AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY 1
            HAVING COUNT(rating_id) >= 50
            ),
            t2 AS (
            SELECT
                author
                , author_id
            FROM authors
            )
            SELECT
                author
                , AVG(avg_rating) AS avg_rating
                , SUM(rating_count) AS rating_count
            FROM books AS b
            RIGHT JOIN t1 on b.book_id = t1.book_id
            LEFT JOIN t2 on b.author_id = t2.author_id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1
        '''

In [13]:
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,author,avg_rating,rating_count
0,J.K. Rowling/Mary GrandPré,4.283844,310.0


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

###  <a id="users_reviews">Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.</a>

In [14]:
query = '''
            SELECT
                ROUND(AVG(review_counts)) AS avg_reviews
            FROM
                (SELECT
                    COUNT(review_id) AS review_counts
                FROM
                    (SELECT
                        username
                    FROM ratings
                    GROUP BY 1
                    HAVING COUNT(rating_id) > 48) AS t1
                LEFT JOIN reviews ON t1.username = reviews.username
                GROUP BY t1.username) AS t2
        '''

In [15]:
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,avg_reviews
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, - 24.

## <a id="conclusion">Общий вывод.</a>

Провели анализ базы данных SQL:
- Количество книг, выпущенных после 1 января 2000 года, - 821.
- Посчитали количество обзоров (999) и среднюю оценку для каждой книги. 
- Определили издательство, которое выпустило наибольшее число книг толще 50 страниц, - `Penguin Books` (42 книги).
- Определили автора с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré со средним рейтингом 4.28, составленным по 310 оценкам пользователей.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, - 24.